# Definition of the project

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"

SPARQLQuery: "SQL Query to run"

SPARQLResult: "Result of the SQLQuery"


Answer: "Final answer here"

Only use the following tables:

In [1]:
!pip install transformers

# Libraries

In [2]:
#define the libraries here!

In [2]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [3]:
import json

# Read Data

In [4]:
with open("questions.json", "r") as fquestions:
  questions = json.load(fquestions)
questions = questions["questions"]

In [5]:
with open("answers.json","r") as fanswers:
  answers = json.load(fanswers)

In [6]:
answers = answers["answers"]


# Dataset

In [10]:
# <s>[INST] <<SYS>>
# System prompt
# <</SYS>>

# User prompt [/INST] Model answer </s>

In [12]:
del answer_str
del system_prompt
del user_prompt
del model_ans

In [ ]:
# generate a prompt from training data
train_prompts = []
answer_list = []

for i, question in enumerate(questions):
  question_string = question["question"]["string"]
  paraphrased_question = question["paraphrased_question"]["string"]
  query = question["query"]["sparql"]
  entities = question["entities"][0]
  relations = question["relations"][0]
  for answer in answers:
    if "results" in answer["answer"].keys():
      for answer  in answer["answer"]["results"]["bindings"]:
        if "answer" in answer.keys():
          answer_list.append(answer["answer"]["value"])

  answer_str = " ".join(answer_list)
  system_prompt = "The natural langue question: " + question_string +"\n, and the paraphrased question: " + paraphrased_question+  "\n   the entity in questions is " + entities + "\n the relation are " + relations

  user_prompt =  " Its SPARQL query is  " + query
  model_ans = "Answers of the SPARQL query are " +answer_str+" when it was run"
  prompt = "<s> [INST] <<SYS>>"+system_prompt+"<</SYS>>\n"+user_prompt+" [/INST] "+ model_ans+" </s>"

  train_prompts.append(prompt)
  del answer_str
  del system_prompt
  del user_prompt
  del model_ans


In [ ]:
!pip install -q datasets
!huggingface-cli login

In [ ]:
from datasets import load_dataset
import re

# Load the dataset
# dataset = load_dataset('timdettmers/openassistant-guanaco')

# # Shuffle the dataset and slice it
# dataset = dataset['train'].shuffle(seed=42).select(range(1000))

# # Define a function to transform the data
# def transform_conversation(example):
#     conversation_text = example['text']
#     segments = conversation_text.split('###')

#     reformatted_segments = []

#     # Iterate over pairs of segments
#     for i in range(1, len(segments) - 1, 2):
#         human_text = segments[i].strip().replace('Human:', '').strip()

#         # Check if there is a corresponding assistant segment before processing
#         if i + 1 < len(segments):
#             assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

#             # Apply the new template
#             reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
#         else:
#             # Handle the case where there is no corresponding assistant segment
#             reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

#     return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)


In [ ]:

transformed_dataset.push_to_hub("guanaco-llama2-1k")

# Model

In [ ]:
# https://huggingface.co/blog/llama2
